In [1]:
cd /orange/adamginsburg/salt/sanhueza/G5.89/

/orange/adamginsburg/salt/sanhueza/G5.89


In [2]:
ls

edgeondisk.reg
G5.89-0.37.config8.allchannels_avg.selfcal_0.5_hogbom.image.fits
G5.89-0.37.config8.spw0.robust0.5.image_cb_small.fits
G5.89-0.37.config8.spw0.robust0.5.image_small.fits
G5.89-0.37.config8.spw1.robust0.5.image_cb_small.fits
G5.89-0.37.config8.spw1.robust0.5.image_small.fits*
G5.89-0.37.config8.spw2.robust0.5.image_cb_small.fits
G5.89-0.37.config8.spw2.robust0.5.image_small.fits
G5.89-0.37.config8.spw3.robust0.5.image_cb_small.fits
G5.89-0.37.config8.spw3.robust0.5.image_small.fits
G5.89.zip
ipython_log_2021-10-01.py
NaCl_v_le_3_stackcube.fits
spectra/


In [3]:
%matplotlib inline
import pylab as pl
from spectral_cube import SpectralCube
from astropy import units as u
from astropy.visualization import quantity_support
quantity_support()

<astropy.visualization.units.quantity_support.<locals>.MplQuantityConverter at 0x2ba436f19be0>

In [4]:
from astropy.table import Table
from astropy import table

In [5]:
import pprint

In [6]:
import scipy, scipy.ndimage

In [7]:
import numpy as np
import os
import glob
from astropy import constants, units as u, table, stats, coordinates, wcs, log, coordinates as coord
import radio_beam
import pyspeckit
import pylab as pl
from astroquery.splatalogue.utils import minimize_table as mt
import sys
sys.path.append('/orange/adamginsburg/salt/Orion_ALMA_2016.1.00165.S/analysis')
import lines
import paths
from salt_tables import (salt_tables, salt_table_names, SO, SO2, HCl, sis_tables, AlCl, AlF, Al37Cl,
                         NaF, AlO, AlOH, NaCN, CaS, CaO)

In [8]:
import regions
from astropy import units as u, coordinates
#reg = regions.Regions.read('big.reg')
regsm = regions.Regions.read('edgeondisk.reg')
#egvsm = regions.Regions.read('sourceAsmaller.reg')
#regvvsm = regions.Regions.read('.reg')

# NaCl Stackl

In [9]:
from spectral_cube.analysis_utilities import stack_cube

In [10]:
nacltables = salt_tables[3:5]

In [11]:
import spectral_cube.analysis_utilities
import imp
imp.reload(spectral_cube.analysis_utilities)
from spectral_cube.analysis_utilities import stack_cube

In [12]:
flist = sorted(glob.glob('G5.89-0.37.config8.spw[0-3].robust0.5.image_small.fits'))

In [13]:
for fn in flist:
    outfile = fn.replace("_small","_cb_small")
    if not os.path.exists(outfile):
        cube = SpectralCube.read(fn, use_dask=True)#.subcube_from_regions(reg)
        cube.allow_huge_operations=True
        cb = cube.beams.common_beam(max_iter=20, max_epsilon=0.01)
        scube = cube.convolve_to(cb)
        scube.to(u.K).write(outfile, overwrite=True)

In [14]:
cbflist = sorted((glob.glob('G5.89-0.37.config8.spw[0-3].robust0.5.image_cb_small.fits')))
cubes = [SpectralCube.read(fn).subcube_from_regions(regsm).to(u.K) for fn in cbflist]
cubes

[SpectralCube with shape=(1920, 49, 80) and unit=K:
  n_x:     80  type_x: RA---SIN  unit_x: deg    range:   270.126048 deg:  270.126288 deg
  n_y:     49  type_y: DEC--SIN  unit_y: deg    range:   -24.068077 deg:  -24.067944 deg
  n_s:   1920  type_s: FREQ      unit_s: Hz     range: 233565132698.900 Hz:235439188519.174 Hz,
 SpectralCube with shape=(1920, 49, 80) and unit=K:
  n_x:     80  type_x: RA---SIN  unit_x: deg    range:   270.126048 deg:  270.126288 deg
  n_y:     49  type_y: DEC--SIN  unit_y: deg    range:   -24.068077 deg:  -24.067944 deg
  n_s:   1920  type_s: FREQ      unit_s: Hz     range: 231065109664.600 Hz:232939165484.874 Hz,
 SpectralCube with shape=(1920, 49, 80) and unit=K:
  n_x:     80  type_x: RA---SIN  unit_x: deg    range:   270.126048 deg:  270.126288 deg
  n_y:     49  type_y: DEC--SIN  unit_y: deg    range:   -24.068077 deg:  -24.067944 deg
  n_s:   1920  type_s: FREQ      unit_s: Hz     range: 216864978830.100 Hz:218739034650.374 Hz,
 SpectralCube with sha

In [15]:
# only needed for VRSCs
if hasattr(cubes[0], 'beams'):
    beams = radio_beam.Beams(beams=[cube.beams.common_beam(max_iter=20, max_epsilon=0.01) for cube in cubes])
    cb=beams.common_beam()
else:
    cb = cubes[0].beam

In [16]:
linelist = np.concatenate([data['Freq'][(data['vu'] <= 3) &
                                          np.any([((data['Freq'].quantity > cube.spectral_extrema[0]) &
                                                   (data['Freq'].quantity < cube.spectral_extrema[1]))
                                                 for cube in cubes],
                                                 axis=0)].quantity
                             for data in nacltables])
linenames = np.concatenate( [data['Species'][(data['vu'] <= 3) &
                                          np.any([((data['Freq'].quantity > cube.spectral_extrema[0]) &
                                                   (data['Freq'].quantity < cube.spectral_extrema[1]))
                                                 for cube in cubes],
                                                 axis=0)]
                             for data in nacltables] )
(linelist, linenames)

(<Quantity [217.980231, 219.614936, 232.509977, 234.251912] GHz>,
 array(['23Na-35Clv=2-2 J=17-16', '23Na-35Clv=1-1 J=17-16',
        '23Na-35Clv=1-1 J=18-17', '23Na-35Clv=0-0 J=18-17'], dtype='<U24'))

In [17]:
from astropy import log
log.setLevel('INFO')

In [18]:
cubes

[SpectralCube with shape=(1920, 49, 80) and unit=K:
  n_x:     80  type_x: RA---SIN  unit_x: deg    range:   270.126048 deg:  270.126288 deg
  n_y:     49  type_y: DEC--SIN  unit_y: deg    range:   -24.068077 deg:  -24.067944 deg
  n_s:   1920  type_s: FREQ      unit_s: Hz     range: 233565132698.900 Hz:235439188519.174 Hz,
 SpectralCube with shape=(1920, 49, 80) and unit=K:
  n_x:     80  type_x: RA---SIN  unit_x: deg    range:   270.126048 deg:  270.126288 deg
  n_y:     49  type_y: DEC--SIN  unit_y: deg    range:   -24.068077 deg:  -24.067944 deg
  n_s:   1920  type_s: FREQ      unit_s: Hz     range: 231065109664.600 Hz:232939165484.874 Hz,
 SpectralCube with shape=(1920, 49, 80) and unit=K:
  n_x:     80  type_x: RA---SIN  unit_x: deg    range:   270.126048 deg:  270.126288 deg
  n_y:     49  type_y: DEC--SIN  unit_y: deg    range:   -24.068077 deg:  -24.067944 deg
  n_s:   1920  type_s: FREQ      unit_s: Hz     range: 216864978830.100 Hz:218739034650.374 Hz,
 SpectralCube with sha

In [20]:
#log.setLevel('DEBUG')
vcen = 10 * u.km/u.s # from Patricio
stack, cutouts = stack_cube(cubes,
                   linelist=linelist,
                   vmin=vcen-30*u.km/u.s,
                   vmax=vcen+30*u.km/u.s,
                   average=np.nanmean,
                   convolve_beam=cb,
                   return_cutouts=True)
#log.setLevel('INFO')
stack.write('NaCl_v_le_3_stackcube.fits', overwrite=True)

Mean of empty slice


In [ ]:
stackc = SpectralCube.read('NaCl_v_le_3_stackcube.fits')
stackc

In [ ]:
jtok = stackc.beam.jtok(217*u.GHz)

In [ ]:
stackc = stackc.subcube_from_regions(regsm)
pl.figure(figsize=(14,5))
pl.subplot(1,3,1)
m0 = stackc.moment0()
m0.quicklook()
pl.title("m0")

pl.subplot(1,3,2)
mx = stackc.max(axis=0)
mx.quicklook()
pl.title("mx")
pl.colorbar()
pl.subplot(1,3,3)
msk = mx > 0.00016*jtok
msk = scipy.ndimage.binary_erosion(msk, iterations=1)
msk = scipy.ndimage.binary_dilation(msk, iterations=5)
pl.imshow(msk, interpolation='none', origin='lower')
pl.tight_layout()
pl.figure()


m1 = stackc.with_mask(msk).with_mask(stackc > 0.0005*jtok).moment1()
pl.figure(figsize=(14,6))
ax1 = pl.subplot(1,2,1)
im = ax1.imshow(m1.value, vmin=-10, vmax=50)
pl.colorbar(mappable=im)
mxv = stackc.with_mask(msk).argmax_world(axis=0)
pl.subplot(1,2,2)
pl.imshow(mxv.value, vmin=-10, vmax=50)
pl.colorbar()


# use water
# vmap = mxv.hdu

In [ ]:
# try creating a vmap based on another line
restval = 232.3929*u.GHz 
restval = 232.6867*u.GHz # 'H2Ov2=1_5(5,0)-6(4,3)': 
cube = [c for c in cubes if (c.spectral_extrema[0] < restval) & (c.spectral_extrema[1] > restval)][0]

scube = cube.subcube_from_regions(regsm).with_spectral_unit(u.km/u.s, velocity_convention='radio', rest_value=restval).spectral_slab(-20*u.km/u.s, 20*u.km/u.s)
pl.figure(figsize=(14,5))
pl.subplot(1,3,1)
m0 = scube.moment0()
m0.quicklook()
pl.title("m0")

pl.subplot(1,3,2)
mx = scube.max(axis=0)
mx.quicklook()
pl.title("mx")
pl.colorbar()
pl.subplot(1,3,3)
msk = mx > 0.00016*jtok
msk = scipy.ndimage.binary_erosion(msk, iterations=1)
msk = scipy.ndimage.binary_dilation(msk, iterations=5)
pl.imshow(msk, interpolation='none', origin='lower')
pl.tight_layout()
pl.figure()


m1 = scube.with_mask(msk).with_mask(scube > 0.0005*jtok).moment1()
pl.figure(figsize=(14,6))
ax1 = pl.subplot(1,2,1)
im = ax1.imshow(m1.value, vmin=-10, vmax=10)
pl.colorbar(mappable=im)
mxv = scube.with_mask(msk).argmax_world(axis=0)
pl.subplot(1,2,2)
pl.imshow(mxv.value, vmin=-10, vmax=10)
pl.colorbar()


vmap = mxv.hdu
vmap = m1.hdu
# the rest value selected above was just the line center; we need to assume it's really 35 km/s off
#vmap.data = vmap.data + vcen.value

In [ ]:
import sys
sys.path.append('/orange/adamginsburg/salt/code/')
from overplot_saltlines import overplot_saltlines

In [ ]:
spectra = []
for ii,cube in enumerate(cubes):
    cube.allow_huge_operations=True
    sp = cube.subcube_from_regions(regsm).mean(axis=(1,2))
    pysp = pyspeckit.Spectrum(data=sp, xarr=sp.spectral_axis, header=sp.header)
    spectra.append(pysp)
    pysp.specname = f'spw{ii+1}'

In [ ]:
mkdir -p spectra/figures

In [ ]:
overplot_saltlines(spectra, vcen=vcen, savepath='spectra/figures')

In [ ]:
ls spectra/figures

In [ ]:
from IPython.display import Image,display

In [ ]:
for fn in glob.glob('spectra/figures/color*png'):
    display(Image(fn))

In [ ]:
import glob
import spectral_cube.analysis_utilities
import reproject
from tqdm.notebook import tqdm

for fn in cbflist:
    basename = os.path.split(fn)[-1].split(".fits")[0]
    try:
        cube = SpectralCube.read(fn, use_dask=True)
    except Exception as ex:
        print(ex)
        continue
    print(cube)
    scube = cube.subcube_from_regions(regsm).to(u.K)
    mx = scube.max(axis=0)
    #msk = mx > 0.003*mx.unit
    print("Mean spectrum")
    sp = scube.mean(axis=(1,2))
    rms = sp.mad_std()
    print(f"rms={rms}")
    
    sp.write(f'spectra/{fn.split("/")[-1].replace(".fits","_meanspec.fits")}', overwrite=True)


    # reproject the velocity map into the cube's coordinate system
    vmap_proj,_ = reproject.reproject_interp(vmap,
                                             scube.wcs.celestial,
                                             shape_out=scube.shape[1:])
    vmap_proj = u.Quantity(vmap_proj, u.km/u.s)

    # convert to km/s with arbitrary rest value so 0 km/s can be interpreted
    if scube.spectral_axis.unit.is_equivalent(u.Hz):
        scube = scube.with_spectral_unit(u.km/u.s, velocity_convention='radio', rest_value=scube.spectral_axis.mean())

    print("Stacking")
    # perform the stacking!
    stack = spectral_cube.analysis_utilities.stack_spectra(scube, vmap_proj,
                                                           v0=0.0*u.km/u.s,
                                                           #progressbar=tqdm,
                                                           #num_cores=8
                                                          )
    fstack = stack.with_spectral_unit(u.GHz)

    pl.suptitle(f"{fn}")
    fstack.write(f'spectra/{basename}_stack.fits', overwrite=True)

    pl.clf()
    fstack.quicklook(filename=f'spectra/figures/{basename}_stack.pdf')
    fstack.quicklook(filename=f'spectra/figures/{basename}_stack.png')
    
    
    fig = pl.figure()
    mappable = pl.subplot(1,2,1).imshow(mx.value, origin='lower', interpolation='none')
    pl.colorbar(mappable)
    pl.subplot(1,2,2)
    pl.suptitle(fn)
    sp.quicklook()

In [ ]:
ls spectra

In [ ]:
stackspectra = []
for ii,fn in enumerate(glob.glob('spectra/*_stack.fits')):
    pysp = pyspeckit.Spectrum(fn)
    basename = os.path.basename(fn)
    spl = basename.split(".")
    pysp.specname = f'{".".join(spl[0:3])}_{spl[4]}_stack.fits'
    print(pysp.specname)
    stackspectra.append(pysp)

In [ ]:
overplot_saltlines(stackspectra, vcen=vcen, savepath='spectra/figures')

In [ ]:
ls spectra/figures/

In [ ]:
for fn in glob.glob('spectra/figures/color*stack.png'):
    display(Image(fn))